# TabSyn Population
Generate synthetic data with TabSyn and a subset from the population

# Import packages

In [1]:
# Standard
import sys

sys.path.append("..")
import json
import shutil
import warnings
from pathlib import Path

# 3rd party
import pandas as pd
import torch
from torch.utils.data import DataLoader

# Local
from src.externals.MIDSTModels.midst_models.single_table_TabSyn.scripts.process_dataset import (
    process_data,
)
from src.externals.MIDSTModels.midst_models.single_table_TabSyn.src import load_config
from src.externals.MIDSTModels.midst_models.single_table_TabSyn.src.data import (
    TabularDataset,
    preprocess,
)
from src.externals.MIDSTModels.midst_models.single_table_TabSyn.src.tabsyn.pipeline import (
    TabSyn,
)
from src.utils import standard

# 1.Configuration

In [2]:
# Paths
data_config_file_path = (
    Path("..")
    / "src"
    / "externals"
    / "MIDSTModels"
    / "midst_models"
    / "single_table_TabSyn"
    / "data_info"
)

model_config_file_path = (
    Path("..")
    / "src"
    / "externals"
    / "MIDSTModels"
    / "midst_models"
    / "single_table_TabSyn"
    / "src"
    / "configs"
)

input_dir = Path(
    "/data8/projets/dev_synthetic_data/data/MIDST/tabsyn_black_box/population/"
)
output_all_dir = (
    input_dir / "all"
)  # Ouput folder to store the results from preprocessing the whole population
output_dir = input_dir / "synth"

# Real population and training data
real_all = "real_all_no_id.csv"
real_train = "real_train_no_id.csv"

# 2.Preprocess data
Prepreprocess both the population and training data

## 2.1 Preprocess population
Preprocess the full table, retain the main table, and pass it as the reference data to preprocess later. This is because the model should have access to all the categories for categorical columns in the data.

In [3]:
# Load the population
df_real_all = pd.read_csv(input_dir / real_all)

In [4]:
df_real_all.shape

(461083, 8)

In [5]:
data_name_all = "trans_all"
data_dir_all = output_all_dir
info_dir_all = data_dir_all / "data_info"
raw_data_dir_all = data_dir_all / "raw_data"
processed_data_dir_all = data_dir_all / "processed_data"

# Create the folders if it doesn't exist
for dir in [
    data_dir_all,
    info_dir_all,
    raw_data_dir_all,
    processed_data_dir_all,
]:
    standard.create_directory(dir)

# Save the population data to data folder
df_real_all.to_csv(raw_data_dir_all / "train.csv", index=False)

# Copy the original config file to the new folder
shutil.copy(data_config_file_path / "trans_all.json", info_dir_all)

# Modify the config file
with open(info_dir_all / "trans_all.json", "r") as file:
    trans_config = json.load(file)

trans_config["name"] = data_name_all
trans_config["data_path"] = str(raw_data_dir_all / "train.csv")
trans_config["test_path"] = ""

# Overwrite the config file
with open(info_dir_all / "trans_all.json", "w") as file:
    json.dump(trans_config, file, indent=4)

In [6]:
process_data(data_name_all, info_dir_all, data_dir_all)

Processing and Saving trans_all Successfully!
Dataset Name: trans_all
Total Size: 461083
Train Size: 456472
Test Size: 4611
Number of Numerical Columns: 4
Number of Categorical Columns: 4


In [7]:
# Save the reference path for later

ref_data_path = processed_data_dir_all / data_name_all

## 2.2 Preprocess training data

In [8]:
# Load the training data

df_real_train = pd.read_csv(input_dir / real_train)

In [9]:
df_real_train.shape

(50000, 8)

In [10]:
data_name = "trans"

data_dir = output_dir / "1st_gen"
info_dir = data_dir / "data_info"
raw_data_dir = data_dir / "raw_data"
processed_data_dir = data_dir / "processed_data"
synth_data_dir = data_dir / "synthetic_data"
model_path = data_dir / "models " / "tabsyn"

# create the new folder if it doesn't exist
for dir in [
    data_dir,
    info_dir,
    raw_data_dir,
    processed_data_dir,
    synth_data_dir,
    model_path,
]:
    standard.create_directory(dir)

# Save the training data to data folder
df_real_train.to_csv(raw_data_dir / "train.csv", index=False)

# Copy the original config file to the new folder
shutil.copy(data_config_file_path / "trans.json", info_dir)

# Modify the config file
with open(info_dir / "trans.json", "r") as file:
    trans_config = json.load(file)

trans_config["name"] = data_name
trans_config["data_path"] = str(data_dir / "raw_data" / "train.csv")
trans_config["test_path"] = ""


# Rewrite the config file
with open(info_dir / "trans.json", "w") as file:
    json.dump(trans_config, file, indent=4)

In [11]:
# Process training data

process_data(data_name, info_dir, data_dir)

Processing and Saving trans Successfully!
Dataset Name: trans
Total Size: 50000
Train Size: 49500
Test Size: 500
Number of Numerical Columns: 4
Number of Categorical Columns: 4


# 3.Build training data

## 3.1 Load model config

In [12]:
raw_config = load_config(model_config_file_path / f"{data_name}.toml")

print(raw_config)

{'task_type': 'regression', 'model_params': {'n_head': 1, 'factor': 32, 'num_layers': 2, 'd_token': 4}, 'transforms': {'normalization': 'quantile', 'num_nan_policy': 'mean', 'cat_nan_policy': None, 'cat_min_frequency': None, 'cat_encoding': None, 'y_policy': 'default'}, 'train': {'vae': {'num_epochs': 4000, 'batch_size': 4096, 'num_dataset_workers': 4}, 'diffusion': {'num_epochs': 10001, 'batch_size': 4096, 'num_dataset_workers': 4}, 'optim': {'vae': {'lr': 0.001, 'weight_decay': 0, 'factor': 0.95, 'patience': 10}, 'diffusion': {'lr': 0.001, 'weight_decay': 0, 'factor': 0.9, 'patience': 20}}}, 'loss_params': {'max_beta': 0.01, 'min_beta': 1e-05, 'lambd': 0.7}}


## 3.2 Make dataset

In [13]:
# Preprocess data
X_num, X_cat, categories, d_numerical = preprocess(
    processed_data_dir / data_name,
    ref_dataset_path=ref_data_path,
    transforms=raw_config["transforms"],
    task_type=raw_config["task_type"],
)

# Separate train and test data
X_train_num, X_test_num = X_num
X_train_cat, X_test_cat = X_cat

# Convert to float tensor
X_train_num, X_test_num = (
    torch.tensor(X_train_num).float(),
    torch.tensor(X_test_num).float(),
)
X_train_cat, X_test_cat = torch.tensor(X_train_cat), torch.tensor(X_test_cat)

# Create dataset object with train data and return tokens of a single row at a time
train_data = TabularDataset(X_train_num.float(), X_train_cat)

# Move test data to gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_test_num = X_test_num.float().to(device)
X_test_cat = X_test_cat.to(device)

# Create train dataloader
train_loader = DataLoader(
    train_data,
    batch_size=raw_config["train"]["vae"]["batch_size"],
    shuffle=True,
    num_workers=raw_config["train"]["vae"]["num_dataset_workers"],
)

No NaNs in numerical features, skipping
No NaNs in numerical features, skipping


# 4.Train model

## 4.1 Instantiate model

In [14]:
tabsyn = TabSyn(
    train_loader,
    X_test_num,
    X_test_cat,
    num_numerical_features=d_numerical,
    num_classes=categories,
    device=device,
)

## 4.2 Train VAE model

In [15]:
# Instantiate VAE model for training

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    tabsyn.instantiate_vae(
        **raw_config["model_params"], optim_params=raw_config["train"]["optim"]["vae"]
    )

Successfully instantiated VAE model.


In [16]:
standard.create_directory(model_path / data_name / "vae")

tabsyn.train_vae(
    **raw_config["loss_params"],
    num_epochs=raw_config["train"]["vae"]["num_epochs"],
    save_path=model_path / data_name / "vae",
)

Epoch 1/4000: 100%|██████████████████████████████████████| 13/13 [00:00<00:00, 13.08it/s]


epoch: 0, beta = 0.010000, Train MSE: 4.131272, Train CE:2.152141, Train KL:0.524534, Val MSE:2.186064, Val CE:2.108609, Train ACC:0.303879, Val ACC:0.285500


Epoch 2/4000: 100%|██████████████████████████████████████| 13/13 [00:00<00:00, 22.17it/s]


epoch: 1, beta = 0.010000, Train MSE: 1.423580, Train CE:2.029743, Train KL:0.736697, Val MSE:0.871317, Val CE:1.861215, Train ACC:0.323276, Val ACC:0.313000


Epoch 3/4000: 100%|██████████████████████████████████████| 13/13 [00:00<00:00, 26.21it/s]


...

epoch: 3997, beta = 0.000008, Train MSE: 0.000006, Train CE:0.000000, Train KL:3.380360, Val MSE:0.000009, Val CE:0.000000, Train ACC:1.000000, Val ACC:1.000000


Epoch 3999/4000: 100%|███████████████████████████████████| 13/13 [00:00<00:00, 45.63it/s]


epoch: 3998, beta = 0.000008, Train MSE: 0.000006, Train CE:0.000000, Train KL:3.380354, Val MSE:0.000009, Val CE:0.000000, Train ACC:1.000000, Val ACC:1.000000


Epoch 4000/4000: 100%|███████████████████████████████████| 13/13 [00:00<00:00, 42.34it/s]

epoch: 3999, beta = 0.000008, Train MSE: 0.000006, Train CE:0.000000, Train KL:3.380344, Val MSE:0.000009, Val CE:0.000000, Train ACC:1.000000, Val ACC:1.000000
Training time: 23.8928 mins
Successfully trained and saved the VAE model!


In [17]:
# Embed all inputs in the latent space

tabsyn.save_vae_embeddings(
    X_train_num, X_train_cat, vae_ckpt_dir=model_path / data_name / "vae"
)

Successfully saved pretrained embeddings on disk!


## 4.3 Train diffusion model

In [18]:
# Load latent space embeddings
train_z, token_dim = tabsyn.load_latent_embeddings(
    model_path / data_name / "vae"
)  # train_z dim: B x in_dim

# Normalize embeddings
mean = train_z.mean(0)
latent_train_data = (train_z - mean) / 2

# Create data loader
latent_train_loader = DataLoader(
    latent_train_data,
    batch_size=raw_config["train"]["diffusion"]["batch_size"],
    shuffle=True,
    num_workers=raw_config["train"]["diffusion"]["num_dataset_workers"],
)

In [19]:
# Instantiate diffusion model for training

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    tabsyn.instantiate_diffusion(
        in_dim=train_z.shape[1],
        hid_dim=train_z.shape[1],
        optim_params=raw_config["train"]["optim"]["diffusion"],
    )

MLPDiffusion(
  (proj): Linear(in_features=32, out_features=1024, bias=True)
  (mlp): Sequential(
    (0): Linear(in_features=1024, out_features=2048, bias=True)
    (1): SiLU()
    (2): Linear(in_features=2048, out_features=2048, bias=True)
    (3): SiLU()
    (4): Linear(in_features=2048, out_features=1024, bias=True)
    (5): SiLU()
    (6): Linear(in_features=1024, out_features=32, bias=True)
  )
  (map_noise): PositionalEmbedding()
  (time_embed): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): SiLU()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
  )
)
The number of parameters: 10559520
Successfully instantiated diffusion model.


In [20]:
# Train diffusion model

tabsyn.train_diffusion(
    latent_train_loader,
    num_epochs=raw_config["train"]["diffusion"]["num_epochs"],
    ckpt_path=model_path / data_name,
)

Epoch 1983/10001: 100%|██████████████████████| 13/13 [00:00<00:00, 37.96it/s, Loss=0.193]


Early stopping
Time:  706.0019540786743


# 5 Generate synthetic data

In [21]:
# Load data info file
with open(processed_data_dir / data_name / "info.json", "r") as file:
    data_info = json.load(file)

data_info["token_dim"] = token_dim

# Get inverse tokenizers
_, _, categories, d_numerical, num_inverse, cat_inverse = preprocess(
    processed_data_dir / data_name,
    ref_dataset_path=ref_data_path,
    transforms=raw_config["transforms"],
    task_type=raw_config["task_type"],
    inverse=True,
)

standard.create_directory(synth_data_dir / data_name)

# Sample data
num_samples = 75000
in_dim = train_z.shape[1]
mean_input_emb = train_z.mean(0)
tabsyn.sample(
    num_samples,
    in_dim,
    mean_input_emb,
    info=data_info,
    num_inverse=num_inverse,
    cat_inverse=cat_inverse,
    save_path=synth_data_dir / data_name / "tabsyn.csv",
)

No NaNs in numerical features, skipping
No NaNs in numerical features, skipping
Time: 12.576858758926392
Saving sampled data to /data8/projets/dev_synthetic_data/data/MIDST/tabsyn_black_box/population/synth/1st_gen/synthetic_data/trans/tabsyn.csv


In [22]:
df_synth = pd.read_csv(synth_data_dir / data_name / "tabsyn.csv")

In [23]:
# Type convertion

col_type = {
    "float": ["amount", "balance"],
    "int": [
        "trans_date",
        "account",
        "trans_type",
        "operation",
        "k_symbol",
        "bank",
    ],
}

df_synth = standard.trans_type(df=df_synth, col_type=col_type, decimal=1)

In [24]:
# Save to file

df_synth.to_csv(output_dir / "1st_gen" / "all_1st_gen.csv", index=False)